<a href="https://colab.research.google.com/github/paramashiva123/machine-learning/blob/main/keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import numpy as np
import pandas as pd

In [79]:
df = pd.read_csv('diabetes.csv')

In [80]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [81]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [82]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [83]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [84]:
x = scaler.fit_transform(x)

In [85]:
x.shape

(768, 8)

In [86]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [87]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [88]:
#1. how to select appropriate optimizer
#2. No.of nodes in a layer
#3. how to select no.of layers
#4. All in all one model

In [89]:
pip install -U keras-tuner --upgrade

In [90]:
import keras_tuner as kt

In [91]:
def build_model(hp):
  model = Sequential()

  counter = 0
  for i in range(hp.Int('num_layers',1,10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),8,128,8),
                      hp.Choice('activation'+ str(i),values=['relu','tanh','sigmoid']),
                      input_dim=x.shape[1]))
      model.add(Dropout(hp.Choice('dropout'+ str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),8,128,8),
                      hp.Choice('activation'+ str(i),values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+ str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','adadelta','sgd','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [92]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='my_dir',
                        project_name='final1')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [93]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 31s


In [94]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 96,
 'activation0': 'relu',
 'dropout0': 0.7,
 'optimizer': 'sgd',
 'units1': 80,
 'activation1': 'relu',
 'dropout1': 0.3,
 'units2': 128,
 'activation2': 'tanh',
 'dropout2': 0.1,
 'units3': 48,
 'activation3': 'tanh',
 'dropout3': 0.8,
 'units4': 128,
 'activation4': 'tanh',
 'dropout4': 0.5,
 'units5': 104,
 'activation5': 'relu',
 'dropout5': 0.3,
 'units6': 24,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 112,
 'activation7': 'relu',
 'dropout7': 0.7,
 'units8': 48,
 'activation8': 'relu',
 'dropout8': 0.7}

In [95]:
model =tuner.get_best_models(num_models=1)[0]

In [96]:
model.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6704 - loss: 0.6214 - val_accuracy: 0.7468 - val_loss: 0.5674
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6997 - loss: 0.6075 - val_accuracy: 0.7468 - val_loss: 0.5562
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6953 - loss: 0.6033 - val_accuracy: 0.7662 - val_loss: 0.5480
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7338 - loss: 0.5682 - val_accuracy: 0.7662 - val_loss: 0.5402
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6864 - loss: 0.6038 - val_accuracy: 0.7727 - val_loss: 0.5323
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7215 - loss: 0.5673 - val_accuracy: 0.7792 - val_loss: 0.5251
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6852 - loss: 0.5763 - val_accuracy: 0.7792 - val_loss: 0.5188
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7009 - loss: 0.5653 - val_accuracy